In [1]:
import pandas as pd
from konlpy.tag import Twitter

In [2]:
# 사진 제외 : 87%
# 사진 포함 : 88%

In [3]:
df_naver = pd.read_csv('A_V_Adj_C_df_naver_post_table.csv',sep='\t',encoding='utf-8')
df_dbdbdeep = pd.read_csv('A_V_Adj_C_df_dbdbdeep_post_table.csv',sep='\t',encoding='utf-8')
df_naver_page = pd.read_csv('A_V_Adj_C_df_naver_page_table.csv',sep='\t',encoding='utf-8')
df_dbdbdeep_page = pd.read_csv('A_V_Adj_C_df_dbdbdeep_page_table.csv',sep='\t',encoding='utf-8')

FileNotFoundError: File b'A_V_Adj_C_df_naver_post_table.csv' does not exist

In [4]:
Total = pd.concat([df_naver,df_dbdbdeep])

In [5]:
Origin_Total = Total

In [6]:
Total = Total.dropna(subset=['Structure_tag'])

In [7]:
Total['Tag_only'] = Total['Structure_tag'].apply(lambda x : "".join(x.split('|')))

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
Total = Total.dropna(subset=['Tag_only'])

In [9]:
def tag_preprocessing(tag):
    tag_list = []
    for i in tag.split('><'):
        i = i.replace('<','')
        i = i.replace('>','')
        i = i.replace('/','')
        tag_list.append(i)
    tag_string = ' '.join(tag_list)
    return tag_string

In [10]:
Total['Tag_only'] = Total['Tag_only'].apply(lambda x : tag_preprocessing(x))

In [11]:
Total['only_tag'] = Total['Structure_tag'].apply(lambda x : " img ".join(list(map(lambda x : 'text' if len(x)>3 else '', x.split('<img>')))).strip().replace('  ',' '))

In [12]:
Total = Total[['only_tag','label']]

In [13]:
text = Total['only_tag'].tolist()

In [14]:
y = Total['label'].tolist()

## TFIDF

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(ngram_range=(5,5),max_features=100)
x = v.fit_transform(text)

In [16]:
v.vocabulary_

{'img img img img img': 0,
 'img img img img text': 1,
 'img img img text img': 2,
 'img img text img img': 3,
 'img img text img text': 4,
 'img text img img img': 5,
 'img text img img text': 6,
 'img text img text img': 7,
 'text img img img img': 8,
 'text img img img text': 9,
 'text img img text img': 10,
 'text img text img img': 11,
 'text img text img text': 12}

### TFIDF model save

In [17]:
import pickle
import numpy as np
#Save vectorizer.vocabulary_
pickle.dump(v,open("[Structure_tag]TFIDF_features_100.pkl","wb"))

# Model_load

In [18]:
v_load = pickle.load(open("[Structure_tag]TFIDF_features_100.pkl", "rb"))

In [19]:
def tfidf_vectorizer(Text):
    try:
        return v_load.transform([Text]).toarray().flatten()
    except:
        return ''

In [20]:
x = Total['only_tag'].apply(lambda x : tfidf_vectorizer(x))

# Data analysis

In [21]:
Array = x.tolist()

In [22]:
Label = Total['label']

In [23]:
test_df = pd.DataFrame({'Array':Array,'label':Label})

In [24]:
test_df = test_df.dropna()

In [25]:
test_df.shape

(16260, 2)

In [26]:
from sklearn.utils import shuffle
test_df = shuffle(test_df)

In [27]:
x = test_df['Array'].apply(lambda x: x.flatten())
x = x.tolist()

In [28]:
y = test_df['label'].tolist()

In [29]:
# from sklearn.utils import shuffle
# test = shuffle(test)
# test = test.sample(frac=1)

## Model_Experiment

In [30]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x , y , test_size=0.33, random_state=42)

In [31]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

result = LinearSVC(random_state=0).fit(x_train, y_train)
result.predict(x_test)
print("SVM :",accuracy_score(result.predict(x_test),y_test))

SVM : 0.6820723071188968


In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_test, result.predict(x_test)))

             precision    recall  f1-score   support

          0       0.57      0.76      0.65      2113
          1       0.80      0.63      0.71      3253

avg / total       0.71      0.68      0.69      5366



In [33]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(result, x_test, y_test, cv=5)

scores                                              

array([0.68621974, 0.66852886, 0.68621974, 0.68283582, 0.68936567])

## Gausian NB

In [34]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(x_train, y_train)
clf.predict(x_test)
print("NB :",accuracy_score(clf.predict(x_test),y_test))

NB : 0.6628773760715617


In [35]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(x_test)))

             precision    recall  f1-score   support

          0       0.55      0.84      0.66      2113
          1       0.84      0.55      0.66      3253

avg / total       0.72      0.66      0.66      5366



In [36]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, x, y, cv=5)
scores                                              

array([0.66154319, 0.64575646, 0.64206642, 0.67619926, 0.64718548])

## Randomforest

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(x_train, y_train)

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [38]:
print("RF :",accuracy_score(clf.predict(x_test),y_test))

RF : 0.7568020872158032


In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(x_test)))

             precision    recall  f1-score   support

          0       0.70      0.67      0.68      2113
          1       0.79      0.81      0.80      3253

avg / total       0.76      0.76      0.76      5366



In [40]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, x, y, cv=5)
scores                                              

array([0.76544728, 0.76476015, 0.76353014, 0.75645756, 0.76099662])

## Logsitic Regression

In [41]:
from  sklearn  import  linear_model
logreg = linear_model.LogisticRegression(C=2.0,random_state=42,solver='sag',multi_class='multinomial',warm_start=True)
logreg.fit(x_train, y_train)

LogisticRegression(C=2.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=42, solver='sag',
          tol=0.0001, verbose=0, warm_start=True)

In [42]:
print("RF :",accuracy_score(logreg.predict(x_test),y_test))

RF : 0.6811405143496087


In [43]:
from sklearn.metrics import classification_report
print(classification_report(y_test, logreg.predict(x_test)))

             precision    recall  f1-score   support

          0       0.57      0.77      0.65      2113
          1       0.81      0.62      0.70      3253

avg / total       0.71      0.68      0.68      5366



In [44]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(logreg, x, y, cv=5)
scores                                              

array([0.68552106, 0.67527675, 0.66666667, 0.69557196, 0.67733005])

## Decision Tree 

In [45]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train)

In [46]:
print("DT :",accuracy_score(clf.predict(x_test),y_test))


DT : 0.7651882221393962


In [47]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(x_test)))

             precision    recall  f1-score   support

          0       0.68      0.76      0.72      2113
          1       0.83      0.77      0.80      3253

avg / total       0.77      0.77      0.77      5366



In [48]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, x, y, cv=5)
scores                                              

array([0.77743621, 0.77613776, 0.77829028, 0.77183272, 0.77176253])

## MLP

In [49]:
import time
from sklearn.neural_network import MLPClassifier

In [50]:
mlp_clf = MLPClassifier(
    hidden_layer_sizes=(50,),
    max_iter=35,
    alpha=1e-4,
    solver='sgd',
    verbose=10,
    tol=1e-4,
    random_state=1,
    learning_rate_init=.1
)

In [51]:
start = time.time()
mlp_clf.fit(x_train, y_train)
end = time.time()
print('Time: {:f}s'.format(end-start))

Iteration 1, loss = 0.60566934
Iteration 2, loss = 0.56715569
Iteration 3, loss = 0.55844553
Iteration 4, loss = 0.55160551
Iteration 5, loss = 0.54742812
Iteration 6, loss = 0.54302695
Iteration 7, loss = 0.53718978
Iteration 8, loss = 0.53223864
Iteration 9, loss = 0.52636029
Iteration 10, loss = 0.52181786
Iteration 11, loss = 0.51499838
Iteration 12, loss = 0.50835433
Iteration 13, loss = 0.50261775
Iteration 14, loss = 0.49715771
Iteration 15, loss = 0.49310729
Iteration 16, loss = 0.48946911
Iteration 17, loss = 0.48715942
Iteration 18, loss = 0.48354555
Iteration 19, loss = 0.48020242
Iteration 20, loss = 0.48036056
Iteration 21, loss = 0.47911710
Iteration 22, loss = 0.47662866
Iteration 23, loss = 0.47428443
Iteration 24, loss = 0.47343449
Iteration 25, loss = 0.47312712
Iteration 26, loss = 0.47296721
Iteration 27, loss = 0.47177735
Iteration 28, loss = 0.47532103
Iteration 29, loss = 0.47221786
Iteration 30, loss = 0.47083346
Iteration 31, loss = 0.47239735
Iteration 32, los

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (35) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [52]:
y_pred = mlp_clf.predict(x_test)
print("테스트 정확도: {:.3f}".format(accuracy_score(y_pred, y_test)))

테스트 정확도: 0.777


In [53]:
from sklearn.metrics import classification_report
print(classification_report(y_test, mlp_clf.predict(x_test)))

             precision    recall  f1-score   support

          0       0.70      0.76      0.73      2113
          1       0.83      0.79      0.81      3253

avg / total       0.78      0.78      0.78      5366



In [54]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(mlp_clf, x, y, cv=5)
scores                                              

Iteration 1, loss = 0.60049288
Iteration 2, loss = 0.56432946
Iteration 3, loss = 0.55663953
Iteration 4, loss = 0.54994737
Iteration 5, loss = 0.54430811
Iteration 6, loss = 0.53946270
Iteration 7, loss = 0.53386508
Iteration 8, loss = 0.52691139
Iteration 9, loss = 0.52429946
Iteration 10, loss = 0.51413413
Iteration 11, loss = 0.51242470
Iteration 12, loss = 0.50152514
Iteration 13, loss = 0.49655239
Iteration 14, loss = 0.49104718
Iteration 15, loss = 0.48832342
Iteration 16, loss = 0.48599433
Iteration 17, loss = 0.48384179
Iteration 18, loss = 0.48962360
Iteration 19, loss = 0.48744220
Iteration 20, loss = 0.48257532
Iteration 21, loss = 0.48498316
Iteration 22, loss = 0.47795181
Iteration 23, loss = 0.47910246
Iteration 24, loss = 0.47670227
Iteration 25, loss = 0.48073357
Iteration 26, loss = 0.47651068
Iteration 27, loss = 0.47536152
Iteration 28, loss = 0.47787040
Iteration 29, loss = 0.47539463
Iteration 30, loss = 0.47551654
Training loss did not improve more than tol=0.000

array([0.72825085, 0.60854859, 0.64790898, 0.61254613, 0.7363888 ])

In [55]:
# import tensorflow as tf
# import random
# import numpy as np

In [56]:
# tf.reset_default_graph()

In [57]:
# # hyper Parameter
# learning_rate = 0.001
# training_epochs = 15
# batch_size = 100

In [58]:
# # input layer
# X = tf.placeholder(tf.float32, [None, 134])
# Y = tf.placeholder(tf.float32, [None, 1])

# # dropout
# keep_prob = tf.placeholder(tf.float32)

# # Xavier_Initializer
# xavier_init = tf.contrib.layers.xavier_initializer()

In [59]:
# # Hidden layers and Output layer
# W1 = tf.get_variable("W1", shape=[134, 67], initializer=xavier_init)
# b1 = tf.Variable(tf.random_normal([67]))
# L1 = tf.nn.relu(tf.matmul(X, W1)+b1)
# dropout1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# W2 = tf.get_variable("W2", shape=[67, 30], initializer=xavier_init)
# b2 = tf.Variable(tf.random_normal([30]))
# L2 = tf.nn.relu(tf.matmul(dropout1, W2)+b2)
# dropout2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# W3 = tf.get_variable("W3", shape=[30, 1], initializer=xavier_init)
# b3 = tf.Variable(tf.random_normal([1]))
# hypothesis = tf.matmul(dropout2, W3)+b3

In [60]:
# # define cost/loss & optimizer
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
# # Initialize
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

In [61]:
# x_train = np.array(x_train)
# x_test = np.array(x_test)
# y_train = np.array(y_train)
# y_test = np.array(y_test)

In [62]:
# x_train[0].reshape(1,134)

In [63]:
# x_train[1].shape

In [64]:
# # Train Model
# for epoch in range(training_epochs):
#     feed_dict = {X:x_train, Y: y_train, keep_prob:0.7}
#     c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
#     avg_cost += c / total_batch

#     print('Epoch:', '{:04d}'.format(epoch +1), 'cost =', '{:.9f}'.format(avg_cost))

# print('Training Finished')

In [65]:
# text_x = [0,0,1,0]
# np.transpose(text_x)